In [7]:
import pandas as pd
import json
import requests

# Looking at ETH Distribution

sources: 
- \[1\] [Ethereum blog: Launching the Ether Sale by Vitalik](https://blog.ethereum.org/2014/07/22/launching-the-ether-sale/)
- \[2\] [Ethereum AMA](https://www.reddit.com/r/IAmA/comments/2bjmgb/hi_we_are_the_ethereum_project_team_ask_us/)
- \[3\] [AVSA Tweet about contributors](https://twitter.com/avsa/status/1210587580741439489)
    

### ICO Details \[1\]:
- tranches:
    - 0-14 days: 2000 ETH per BTC
    - linear decline: 1337 ETH per BTC
- lasts 42 days
- annual issuance is 0.26x initial ETH sold
- two endowment pools (each 0.099x initial ETH sold):
    - early contributors to the project
    - long-term endowment to non-profit foundation
- reserved right to use up to 5000 BTC from pre-sale while pre-sale is running to ramp development
- exodus 3-4 multisig address: `36PrZ1KHYMpqSyAQXSG8VwbUiq2EogxLo`


## Pre-mine Analysis

In [21]:
# Bring in Mainnet Genesis file
with open('./eth_distribution/frontier.json', 'r') as f:
    genesis_json = json.load(f)
genesis_txns = genesis_json['alloc']
genesis_txns = [{"address": key, "balance": value['balance']} for key, value in genesis_txns.items()]
genesis_txns = pd.DataFrame(genesis_txns)
genesis_txns['balance'] = genesis_txns['balance'].apply(float) / 1e18
genesis_txns['address'] = '0x' + genesis_txns['address']
genesis_txns.sort_values(by='balance', ascending=False)

,address,balance
6698,0x5abfec25f74cd88437631a7731906932776356f9,1.190148e+07
2197,0x1937c5c515057553ccbd46d5866455ce66290284,1.000000e+06
7364,0x11172b278ddd44eea2fdf4cb1d16962391c453d9,9.359000e+05
5414,0xff7843c7010aa7e61519b762dfe49124a76b0e4e,9.335800e+05
164,0x5281733473e00d87f11e9955e589b59f4ac28e7a,6.603600e+05
...,...,...
8166,0xf2c2904e9fa664a11ee25656d8fd2cc0d9a522a0,1.337000e+01
1399,0x7641f7d26a86cddb2be13081810e01c9c83c4b20,1.337000e+01
3177,0x33581cee233088c0860d944e0cf1ceabb8261c2e,1.337000e+01
4259,0x5ed3f1ebe2ae6756b5d8dc19cad02c419aa5778b,0.000000e+00


In [44]:
headers = {}
# The GraphQL query (with a few aditional bits included) itself defined as a multi-line string.       
query = """
{
  domains(where: {
    owner: "%s"
  }){
    id
    name
    labelName
    labelhash
  }
}
"""

def run_query(query, address): # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.thegraph.com/subgraphs/name/ensdomains/ens', json={'query': query % (address)}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))

def pull_ens_name(result):
    if len(result['data']['domains']) != 0:
        return result['data']['domains'][0]['name']
    else:
        return ''

def query_and_pull(address):
    return pull_ens_name(run_query(query, address))

# result = query_and_pull('0xb913626032140a86c77d1fdde4f611a00d589c55') # Execute the query
result = query_and_pull('0x5abfec25f74cd88437631a7731906932776356f9')
print(result)

In [45]:
genesis_txns.iloc[:50,:].sort_values(by="balance", ascending=False).address.apply(query_and_pull)

14    
24    
23    
32    
34    
48    
49    
42    
44    
33    
10    
13    
37    
36    
41    
40    
4     
30    
8     
46    
21    
0     
29    
17    
22    
45    
39    
19    
5     
2     
47    
15    
7     
12    
20    
25    
38    
16    
3     
1     
35    
31    
27    
28    
43    
26    
18    
9     
6     
11    
Name: address, dtype: object

### Open contributions

### Contributor Allocations

source: \[3\]
- Vitalik: Ethereum 
- Hoskinson: Classic
- Di'orio: Jaxx
- Mihai: Akasha
- Gavin: Polkadot
- Lubin: Consensys
- Jeff: Not involved anymore
- Amir: seems to be an angel investor now?
- Koblitz: cryptography professor (same as before)
- Ralph Merkle: same as above
- Stephan Tual: Slockit, Atlas Neue
- Taylor: volunteers for swarm team
- Jeremy Wood: Cardano
- Mathias: Braveno (cryptoexchange)
- Ryan: Dash
- Fierro: Mimir (lab building blockchain apps?)
- Stott: couldn’t find
- Meikle: Ethereum
- Colby: vDice
- Sureanu: Akasha
- Hidskes: couldnt find
- Mohan: Themys (blockchain risk assessment?)
- Houwu Chen: seems to be a researcher still?
- Nguyen: couldn’t find. There’s a "Forbes under 30” with his name, same guy?
- Wilding: L4, a web3 studio that built ETHGlobal
- Gluck: Riot games
- Chow: still at ethereum I believe?

### Summary